<a href="https://colab.research.google.com/github/roklp/MLP34/blob/main/kaggle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/data/kaggle1/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/kaggle1/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/data/kaggle1/sample_submission.csv')

train.shape, test.shape

((20758, 18), (13840, 17))

In [3]:
train.head(1)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.66995,yes,yes,2.0,2.983297,Sometimes,no,2.763573,no,0.0,0.976473,Sometimes,Public_Transportation,Overweight_Level_II


In [4]:
X_train_id = train.pop('id')
X_test_id = test.pop('id')

In [5]:
y = train.pop('NObeyesdad')

In [6]:
import numpy as np

cat_cols = list(train.select_dtypes(exclude = np.number).columns)
num_cols = list(test.select_dtypes(include = np.number).columns)

print(cat_cols)
print(num_cols)

['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    train, y,
    stratify = y,
    test_size = 0.3,
    random_state=42
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((14530, 16), (6228, 16), (14530,), (6228,))

In [ ]:
# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# modeling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV

# metrics
from sklearn.metrics import f1_score

# utils
# from sklearn.utils.fixes import loguniform
from scipy.stats import loguniform

param_grid = {
    "clf__learning_rate": loguniform(0.001, 0.1),
    "clf__n_estimators": np.arange(50, 500),
    "clf__max_depth": np.arange(3, 30, 3),
}

column_transformer = ColumnTransformer([
    ("scaler", StandardScaler(), num_cols),
    ("ohc_encoder", OneHotEncoder(handle_unknown='ignore'), cat_cols),
], remainder = "passthrough")

pipeline = Pipeline([
    ("preprocessor", column_transformer),
    ("clf", LGBMClassifier(random_state=42, objective='multiclass', num_class=4))
])


random_search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 3,
    scoring="f1_macro",
    cv=2,
    verbose=-1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

In [9]:
# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# modeling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# metrics
from sklearn.metrics import f1_score

# utils
from scipy.stats import loguniform

param_grid = {
    "clf__learning_rate": loguniform(0.001, 0.1),
    "clf__n_estimators": np.arange(50, 500),
    "clf__max_depth": np.arange(3, 30, 3),
}

column_transformer = ColumnTransformer([
    ("scaler", StandardScaler(), num_cols),
    ("ohc_encoder", OneHotEncoder(handle_unknown='ignore'), cat_cols),
], remainder = "passthrough")

pipeline = Pipeline([
    ("preprocessor", column_transformer),
    ("clf", XGBClassifier(objective='multi:softmax', num_class=4, random_state=42))
])

random_search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = param_grid,
    n_iter = 3,
    scoring="f1_macro",
    cv=2,
    verbose=-1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)


ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1471, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6], got ['Insufficient_W